# Day 60: 70 Days Pre-Bootcamp Online Class with Data Science Nigeria - Machine Learning Stream #DSN70daysofML
Testing the network - Training a neural network to play a game with TensorFlow and Open AI <br>

In [1]:
import gym
import random
import numpy as np
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import median, mean
from collections import Counter

LR = 1e-3
env = gym.make("CartPole-v0")
env.reset()
goal_steps = 500
score_requirement = 50
initial_games = 1000

C:\Users\User\anaconda3\envs\gputensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\anaconda3\envs\gputensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\anaconda3\envs\gputensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\anaconda3\envs\gputensorflow\lib\site-

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [2]:
def some_random_games_first():
    # Each of these is its own game.
    for episode in range(5):
        env.reset()
        # this is each frame, up to 200...but we wont make it that far.
        for t in range(200):
            # This will display the environment
            # Only display if you really want to see it.
            # Takes much longer to display it.
            env.render()
            
            # This will just create a sample action in any environment.
            # In this environment, the action can be 0 or 1, which is left or right
            action = env.action_space.sample()
            
            # this executes the environment with an action, 
            # and returns the observation of the environment, 
            # the reward, if the env is over, and other info.
            observation, reward, done, info = env.step(action)
            if done:
                break

In [3]:
some_random_games_first()

In [4]:
def initial_population():
    # [OBS, MOVES]
    training_data = []
    # all scores:
    scores = []
    # just the scores that met our threshold:
    accepted_scores = []
    # iterate through however many games we want:
    for _ in range(initial_games):
        score = 0
        # moves specifically from this environment:
        game_memory = []
        # previous observation that we saw
        prev_observation = []
        # for each frame in 200
        for _ in range(goal_steps):
            # choose random action (0 or 1)
            action = random.randrange(0,2)
            # do it!
            observation, reward, done, info = env.step(action)
            
            # notice that the observation is returned FROM the action
            # so we'll store the previous observation here, pairing
            # the prev observation to the action we'll take.
            if len(prev_observation) > 0 :
                game_memory.append([prev_observation, action])
            prev_observation = observation
            score+=reward
            if done: break

        # IF our score is higher than our threshold, we'd like to save
        # every move we made
        # NOTE the reinforcement methodology here. 
        # all we're doing is reinforcing the score, we're not trying 
        # to influence the machine in any way as to HOW that score is 
        # reached.
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                # convert to one-hot (this is the output layer for our neural network)
                if data[1] == 1:
                    output = [0,1]
                elif data[1] == 0:
                    output = [1,0]
                    
                # saving our training data
                training_data.append([data[0], output])

        # reset env to play again
        env.reset()
        # save overall scores
        scores.append(score)
    
    # just in case you wanted to reference later
    training_data_save = np.array(training_data)
    np.save('saved.npy',training_data_save)
    
    # some stats here, to further illustrate the neural network magic!
    print('Average accepted score:',mean(accepted_scores))
    print('Median score for accepted scores:',median(accepted_scores))
    print(Counter(accepted_scores))
    
    return training_data

In [5]:
initial_population()

C:\Users\User\anaconda3\envs\gputensorflow\lib\site-packages\gym\logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
C:\Users\User\anaconda3\envs\gputensorflow\lib\site-packages\ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Average accepted score: 62.36842105263158
Median score for accepted scores: 59.0
Counter({53.0: 5, 60.0: 3, 51.0: 3, 58.0: 3, 56.0: 2, 55.0: 2, 72.0: 2, 50.0: 2, 61.0: 2, 52.0: 1, 67.0: 1, 64.0: 1, 68.0: 1, 80.0: 1, 54.0: 1, 76.0: 1, 79.0: 1, 110.0: 1, 78.0: 1, 69.0: 1, 70.0: 1, 77.0: 1, 66.0: 1})


[[array([-0.02566796, -0.17890173,  0.03087715,  0.3063103 ]), [0, 1]],
 [array([-0.029246  ,  0.01576693,  0.03700336,  0.02352294]), [1, 0]],
 [array([-0.02893066, -0.17986561,  0.03747381,  0.32764737]), [1, 0]],
 [array([-0.03252797, -0.37550047,  0.04402676,  0.63190837]), [0, 1]],
 [array([-0.04003798, -0.18101953,  0.05666493,  0.35340933]), [0, 1]],
 [array([-0.04365837,  0.0132528 ,  0.06373312,  0.07911911]), [1, 0]],
 [array([-0.04339331, -0.18272214,  0.0653155 ,  0.39120957]), [0, 1]],
 [array([-0.04704776,  0.01141498,  0.07313969,  0.11981392]), [1, 0]],
 [array([-0.04681946, -0.18467452,  0.07553597,  0.43464571]), [1, 0]],
 [array([-0.05051295, -0.38078003,  0.08422888,  0.75015164]), [0, 1]],
 [array([-0.05812855, -0.18691455,  0.09923191,  0.48511837]), [0, 1]],
 [array([-0.06186684,  0.00667731,  0.10893428,  0.22528545]), [0, 1]],
 [array([-0.06173329,  0.20008742,  0.11343999, -0.03114669]), [1, 0]],
 [array([-0.05773154,  0.00353689,  0.11281706,  0.29506402]), [

In [6]:
def neural_network_model(input_size):

    network = input_data(shape=[None, input_size, 1], name='input')

    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
    model = tflearn.DNN(network, tensorboard_dir='log')

    return model

In [7]:
def train_model(training_data, model=False):

    X = np.array([i[0] for i in training_data]).reshape(-1,len(training_data[0][0]),1)
    y = [i[1] for i in training_data]

    if not model:
        model = neural_network_model(input_size = len(X[0]))
    
    model.fit({'input': X}, {'targets': y}, n_epoch=5, snapshot_step=500, show_metric=True, run_id='openai_learning')
    return model

In [8]:
training_data = initial_population()

Average accepted score: 60.875
Median score for accepted scores: 58.5
Counter({59.0: 3, 52.0: 3, 58.0: 2, 54.0: 2, 51.0: 2, 53.0: 2, 64.0: 2, 50.0: 2, 63.0: 2, 71.0: 1, 81.0: 1, 77.0: 1, 76.0: 1, 57.0: 1, 61.0: 1, 87.0: 1, 55.0: 1, 66.0: 1, 80.0: 1, 56.0: 1, 62.0: 1})


C:\Users\User\anaconda3\envs\gputensorflow\lib\site-packages\ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [9]:
model = train_model(training_data)

Training Step: 149  | total loss: 0.66747 | time: 0.656s
| Adam | epoch: 005 | loss: 0.66747 - acc: 0.6129 -- iter: 1856/1916
Training Step: 150  | total loss: 0.66707 | time: 0.680s
| Adam | epoch: 005 | loss: 0.66707 - acc: 0.6110 -- iter: 1916/1916
--


In [10]:
scores = []
choices = []
for each_game in range(10):
    score = 0
    game_memory = []
    prev_obs = []
    env.reset()
    for _ in range(goal_steps):
        env.render()

        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(model.predict(prev_obs.reshape(-1,len(prev_obs),1))[0])

        choices.append(action)
                
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        game_memory.append([new_observation, action])
        score+=reward
        if done: break

    scores.append(score)

print('Average Score:',sum(scores)/len(scores))
print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))
print(score_requirement)

Average Score: 200.0
choice 1:0.5  choice 0:0.5
50
